In [1]:
import pandas as pd
import json
from google.cloud import storage
import os
import glob
import dat
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import re
from io import StringIO
import newlinejson as nlj
from json import loads, dumps
# import msgspec
import gcsfs

In [5]:
# def hello_gcs(project_id, bucket_name):
#     bq_client = bigquery.Client(project_id)
#     bucket = storage.Client().bucket(bucket_name)
#     for blob in bucket.list_blobs(prefix=""):
#         if ".json" in blob.name: #Checking for json blobs as list_blobs also returns folder_name
#         #    print(blob.name)
#            job_config = bigquery.LoadJobConfig(
#             autodetect=True,
#             source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON)
#            json_filename = blob.name.split('.json')[0]
#         #    json_filename = re.findall(r".*/(.*).json",blob.name) #Extracting file name for BQ's table id
#         #    print(json_filename)
#            table_id = f"capable-memory-417812.premiership.{json_filename}" # Determining table name
#            print(table_id)
#         #    try: #Check if the table already exists and skip uploading it.
#         #        bq_client.get_table(table_id)
#         #        print("Table {} already exists. Not uploaded.".format(bq_table_id))
#         #    except NotFound: #If table is not found, upload it.    
#            uri = f"gs://{bucket_name}/{blob.name}"
#            print(uri)
#            load_job = bq_client.load_table_from_uri(
#                uri, table_id, location = 'europe-west8',  
#                job_config=job_config,
#            )  # Make an API request.
#            load_job.result()  # Waits for the job to complete.
#            destination_table = bq_client.get_table(table_id)  # Make an API request.
#            print("Loaded {} rows.".format(destination_table.num_rows))

In [ ]:
# hello_gcs('capable-memory-417812',"game_data_giray_jsonl_conv")

In [10]:
# df = pd.read_json('gs://game_data_giray/epl_2022_2023_07_02_2023.json')
# df = pd.read_json('gs://game_data_giray/premiership_short.json')

In [11]:
def list_blob_elements(bucket_name):
    """Lists blob elements in mentioned bucket"""

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blobs = storage_client.list_blobs(bucket_name)
    # blob = bucket.blob(blob_name)
    list_blobs = []
   
    for blob in blobs:
        list_blobs.append(blob.name)
        
    return list_blobs

In [13]:
list_blob_elements("game_data_giray_jsonl_conv")

['epl_2022_2023_07_02_2023.json_nl.json',
 'epl_2022_2023_07_02_2024.json_nl.json']

In [2]:
def crate_new_bucket(bucket_name, project_id):
    # Instantiating storage class
    storage_client = storage.Client(project_id)
    # The name for the new bucket gathering from existing bucket
    bucket_name_conv = f"{bucket_name}_jsonl_conv" # + "jsonl_conv"
    print(bucket_name_conv)
    # Creates the new bucket if bucket already exists using existing name with conversion
    if storage_client.bucket(bucket_name_conv).exists():
        print('eben')
        bucket_new  = bucket_name_conv
    # If bucket does not exists crates it and gets its name
    else:
        bucket_new = storage_client.create_bucket(bucket_name_conv, location='europe-west8')
        bucket_new.location = 'europe-west8'
        print("created bucket {} in {}".format(bucket_new.name, bucket_new.location))
        bucket_new  = bucket_new.name
    return bucket_new


def convert_json_jsonl(bucket_name, project_id, blob_name = [], path = []):
    ''' This function reads multiple json files from location passed as parameter 
    and converts them to jsonl and saves into created bucket with an inner function
    if bucket exists uses existing bucket'''
    # New bucket creation
    new_bucket = crate_new_bucket(bucket_name,project_id)

    # Instantiating needed classes and methods
    storage_client = storage.Client(project_id)
    bucket = storage_client.bucket(bucket_name)
    blobs = storage_client.list_blobs(bucket_name)
    
    # Loop through blobs in bucket and converting nljson with saving into new bucket path
    for blob in blobs:
        print(blob.name)
        df = pd.read_json("gs://{}/{}".format(bucket_name, blob.name)) 
        df.to_json(f'gs://{new_bucket}/{blob.name}',orient="records",lines=True)
    return 'json_files_converted_nljson'

In [3]:
convert_json_jsonl("game_data_giray",'capable-memory-417812')

game_data_giray_jsonl_conv


/tmp/ipykernel_9412/1572037636.py:14: DeprecationWarning: Assignment to 'Bucket.location' is deprecated, as it is only valid before the bucket is created. Instead, pass the location to `Bucket.create`.
  bucket_new.location = 'europe-west8'


created bucket game_data_giray_jsonl_conv in europe-west8
epl_2022_2023_07_02_2023.json
epl_2022_2023_07_02_2024.json


'json_files_converted_nljson'